<a href="https://colab.research.google.com/github/Dasha-nz/nz/blob/main/%D0%9A.%D0%9B./%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import time
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# Примеры текстов
examples = [
    "вот в инете откапал такую инеерсную статейку предлагаю вашему внимани",
    "может и в правду лутше тебе молчать чем пытаться сказать",
    "утром мы сидели как сычи а потом каааак начали работать"
]

# 1. Измерение времени работы встроенных методов
def measure_builtin_methods():
    import difflib

    results = {}
    start_time = time.time()
    for i in range(len(examples)):
        for j in range(i + 1, len(examples)):
            difflib.SequenceMatcher(None, examples[i], examples[j]).ratio()
    elapsed_time = time.time() - start_time
    results['difflib'] = elapsed_time

    return results

# 2. Функция Дамерау-Левенштейна
def damerau_levenshtein(s1, s2):
    len_s1, len_s2 = len(s1), len(s2)
    d = np.zeros((len_s1 + 1, len_s2 + 1), dtype=int)

    for i in range(len_s1 + 1):
        d[i][0] = i
    for j in range(len_s2 + 1):
        d[0][j] = j

    for i in range(1, len_s1 + 1):
        for j in range(1, len_s2 + 1):
            cost = 0 if s1[i - 1] == s2[j - 1] else 1
            d[i][j] = min(
                d[i - 1][j] + 1,  # Удаление
                d[i][j - 1] + 1,  # Вставка
                d[i - 1][j - 1] + cost  # Замена
            )
            if i > 1 and j > 1 and s1[i - 1] == s2[j - 2] and s1[i - 2] == s2[j - 1]:
                d[i][j] = min(d[i][j], d[i - 2][j - 2] + cost)  # Транспозиция

    return d[len_s1][len_s2]

# Измерение времени работы функции Дамерау-Левенштейна
def measure_damerau_levenshtein():
    start_time = time.time()
    for i in range(len(examples)):
        for j in range(i + 1, len(examples)):
            damerau_levenshtein(examples[i], examples[j])
    elapsed_time = time.time() - start_time
    return elapsed_time

# 3. Функция генерации опечаток
def generate_typos(word):
    keyboard_layout = {
        'ф': ['ы', 'а', 'с'], 'ы': ['ф', 'в', 'с', 'д'], 'в': ['ы', 'е', 'д', 'ж'],
        'е': ['в', 'н', 'ж', 'з'], 'н': ['е', 'г', 'з', 'х'], 'г': ['н', 'й', 'х', 'ъ'],
        'й': ['г', 'ц', 'ъ'], 'ц': ['й', 'у'], 'у': ['ц', 'к'],
        'к': ['у', 'е'], 'е': ['к', 'н', 'и'], 'н': ['е', 'г', 'т'],
        'г': ['н', 'ш', 'ь'], 'ш': ['г', 'щ'], 'щ': ['ш'],
        'а': ['ф', 'п', 'я'], 'п': ['а', 'р', 'я'], 'р': ['п', 'о', 'я'],
        'о': ['р', 'л', 'я'], 'л': ['о'], 'я': ['а', 'п', 'о'],
        'з': ['ж', 'х'], 'ж': ['з', 'ч'], 'ч': ['ж', 'ш'], 'ш': ['ч']
    }

    typos = []
    for i in range(len(word)):
        if word[i] in keyboard_layout:
            for typo in keyboard_layout[word[i]]:
                typos.append(word[:i] + typo + word[i + 1:])
    return typos

# 4. Функция сравнения векторов с использованием CountVectorizer
def compare_vectors():
    vectorizer = CountVectorizer(analyzer='char')
    X = vectorizer.fit_transform(examples)
    return X.toarray()

# Запуск измерений
builtin_methods_time = measure_builtin_methods()
damerau_time = measure_damerau_levenshtein()
typos = generate_typos("пример")
vectors = compare_vectors()

# Вывод результатов
print("Время работы встроенных методов:", builtin_methods_time)
print("Время работы функции Дамерау-Левенштейна:", damerau_time)
print("Сгенерированные опечатки:", typos)
print("Векторы частотности:", vectors)


Время работы встроенных методов: {'difflib': 0.0016019344329833984}
Время работы функции Дамерау-Левенштейна: 0.039070844650268555
Сгенерированные опечатки: ['аример', 'рример', 'яример', 'ппимер', 'поимер', 'пяимер', 'примкр', 'примнр', 'примир', 'примеп', 'примео', 'примея']
Векторы частотности: [[ 9  8  0  4  1  1  7  0  0  4  1  3  2  2  5  2  2  2  2  6  4  0  1  0
   0  3  0]
 [ 9  5  1  2  0  1  5  1  1  1  0  1  2  3  0  2  2  1  2  7  2  2  1  1
   3  0  1]
 [ 9 10  1  0  0  1  1  0  0  4  0  4  2  3  1  4  1  2  2  4  1  2  0  2
   1  0  0]]
